In [5]:
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import calendar
from tqdm import tqdm
from loguru import logger

In [6]:
topic_mapping = {
    'Особое мнение': 'Общество',
    'Технологии': 'Общество',
    'Строительство': 'Город',
    'Недвижимость': 'Город',
    'ЖКХ': 'Город',
    'Авто': 'Город',
    'Финляндия': 'Политика',
    'Власть': 'Политика',
    'Открытое письмо': 'Политика',
    'Бизнес': 'Финансы',
    'Работа': 'Финансы',
    'Новости компаний': 'Финансы',
    'Бизнес-трибуна': 'Финансы',
    'Доктор Питер': 'Здоровье',
    'Туризм': 'Здоровье',
    'Спорт': 'Здоровье',
    'Афиша Plus': 'Образ жизни',
    'Доброе дело': 'Образ жизни'
}


In [7]:
LOGGING = 'logs.log'
FORMAT = "{time} {level} {message}"

In [8]:
def get_page(p) -> list:
    """A function to retrieve data from a web page."""

    logger.add(LOGGING)
    logger.add(LOGGING, format=FORMAT)

    # Generating the url for the request
    url = f'https://www.fontanka.ru/{p}/all.html'
    
    # Receiving a response from the server
    response = requests.get(url)

    # Convert the response to a BeautifulSoup object
    tree = BeautifulSoup(response.content, 'html.parser')
    
    # Find all the news for the day (tag - a string of 5 characters, for example: KXak5)
    news = tree.find_all('li', {'class': re.compile(r'^\w{5}$')})
    
    info = []

    # Going around every news item
    for post in news:

        # Finding the news title
        try:
            title = post.select_one('li div div a').text
        except Exception as e:
            logger.warning(f"Failed to get the news title. Publication date: {p}. Exception: {e}")
            pass

        # Finding the news topic
        try:
            topic = post.find('a').get('title')
        except Exception as e:
            logger.warning(f"Failed to get the news topic. Publication date: {p}. Exception: {e}")
            pass

        # Finding the news link
        try:
            link = post.select_one('li div div a').get('href')
        except Exception as e:
            logger.warning(f"Failed to get the news link. Publication date: {p}. Exception: {e}")
            pass

        # Finding the news time of publication
        try:
            time = post.select_one('li div time span').text
        except Exception as e:
            logger.warning(f"Failed to get the news publication time. Publication date: {p}. Exception: {e}")
            pass
    
        # Finding the news number of comments
        try:
            comm = post.select_one('li div div a span').text
        except Exception as e:
            logger.warning(f"Failed to get the news comments number. Publication date: {p}. Exception: {e}")
            pass

        # Creating the url with our link
        if "https://doctorpiter.ru" in link:
            urli = link
        elif "https://www.fontanka.ru/longreads/" in link:
            urli = link
        else:
            urli = "https://www.fontanka.ru" + link

        # Going inside each news item we found
        response_inner = requests.get(urli)
        tree_inner = BeautifulSoup(response_inner.content, 'html.parser')

        # If this is affiliate news from doctorpiter.ru website
        if "https://doctorpiter.ru" in urli:
            views = 9999 # This site does not have a view counter

            try:
                content = tree_inner.find('section', {'class' : "ds-article-content"}).text
            except Exception as e:
                logger.warning(f"Failed to get the content from doctorpiter. Publication date: {p}. Exception: {e}")
                pass

        else:
            # Such pages are promotional and have no clear markup
            if "https://www.fontanka.ru/longreads/" in urli:
                logger.warning(f"It's the longread. Publication date: {p}.")
                pass

            else:
                # If this is news from fontanka.ru
                if "https://www.fontanka.ru" in urli:
                    try:
                        content = tree_inner.select_one('body div div div div div section div article div section div').text
                    except Exception as e:
                        logger.warning(f"Failed to get the content from fontanka. Publication date: {p}. Exception: {e}")
                        pass

                    try:
                        views = tree_inner.find('div', {'class': re.compile(r'^[A]\dg[a-z]$')}).text
                    except Exception as e:
                        logger.warning(f"Failed to get the viewers number from fontanka. Publication date: {p}. Exception: {e}")
                        pass

                else:
                    logger.error(f"Unknown source of the news post. Publication date: {p}.")

        # Class synthesis
        mapped_topic = topic_mapping.get(topic, topic)

        # Creating a table row with the received data
        row = {
            'date': p,
            'title': title,
            'topic': mapped_topic,
            'url': urli,
            'time': time,
            'comm_num': comm,
            'views': views,
            'content': content
            }

        info.append(row)
    
    return info

In [11]:
infa = []
infa.extend(get_page('2023/11/20'))
df = pd.DataFrame(infa)

In [12]:
df

,date,title,topic,url,time,comm_num,views,content
0,2023/11/20,В Музее политической истории открылась выставк...,Образ жизни,https://www.fontanka.ru/2023/11/20/72930683/,01:29,17,1151,Выставка «Нюрнбергский процесс: взгляд из Моск...
1,2023/11/20,Умерла бывшая первая леди США. Ей было 96 лет,Общество,https://www.fontanka.ru/2023/11/20/72931013/,00:54,15,2530,В штате Джорджия скончалась бывшая первая леди...
2,2023/11/20,"В районе Электростали перехвачен беспилотник, ...",Происшествия,https://www.fontanka.ru/2023/11/20/72931007/,00:12,21,2474,"Беспилотник, направлявшийся в сторону Москву, ..."


In [ ]:
infa = []

for yy in tqdm(range(2022, 2020, -1), desc="Years"):
    for mm in tqdm(range(12, 0, -1), desc="Months", leave=False):
        for dd in tqdm(range(31, 0, -1), desc="Days", leave=False):
            if dd <= calendar.monthrange(yy, mm)[1]:
                p = f'{yy:04}/{mm:02}/{dd:02}'
                try:
                    infa.extend(get_page(p))
                    tqdm.write(f"Processed date: {p}")
                    df = pd.DataFrame(infa)
                    df.to_csv('test.csv', index=False, mode='a')
                    infa = []
                    
                except Exception as e:
                    logger.error(f"Failed to get data for: {p}. Exception: {e}.")
                    pass

In [16]:
df = pd.read_csv('test.csv')

In [17]:
df

,date,title,topic,url,time,comm_num,views,content
0,2022/12/28,"Хмурый, львиноголовый, карликовый. Символ 2023...",Общество,https://www.fontanka.ru/2022/12/28/71938454/,23:59,2,9820,В последние дни нам предлагали ассортимент сим...
1,2022/12/28,"Медведев или Кадыров? Стало известно, какие те...",Общество,https://www.fontanka.ru/2022/12/28/71938472/,23:46,22,11026,Самым популярным телеграм-каналом года среди р...
2,2022/12/28,СКА вновь уступил ЦСКА,Здоровье,https://www.fontanka.ru/2022/12/28/71938469/,23:33,1,5666,Поражением для хоккейного клуба СКА завершился...
3,2022/12/28,«Декриминализовать» нелегальное использование....,Общество,https://www.fontanka.ru/2022/12/28/71938457/,23:19,44,23974,Минцифры готовит законопроект о принудительном...
4,2022/12/28,Когда неудержимо хочется в Шушары. Над городом...,Город,https://www.fontanka.ru/2022/12/28/71936942/,23:05,3,19324,Кадр супервидовой веб-камерыПоделитьсяОдин из ...
...,...,...,...,...,...,...,...,...
95,2022/12/28,На «Госуслугах» открылся прием заявлений на по...,Общество,https://www.fontanka.ru/2022/12/28/71936408/,12:37,2,6832,Семьи с детьми до 17 лет и беременные женщины ...
96,2022/12/28,"ФСБ: Уничтожены двое боевиков, готовивших тера...",Общество,https://www.fontanka.ru/2022/12/28/71936399/,12:28,3,3722,"ФСБ сообщила о ликвидации двоих боевиков, гото..."
97,2022/12/28,Женщина неделю заперта в доме в Приморье из-за...,Происшествия,https://www.fontanka.ru/2022/12/28/71936375/,12:19,2,5423,"После циклона, засыпавшего снегом Приморский к..."
98,2022/12/28,"На Ржевке упал автокран, есть пострадавший",Происшествия,https://www.fontanka.ru/2022/12/28/71936423/,12:15,2,7843,Помощь спасателей потребовалась в промзоне в К...
